# Karim Kuserbaev

## Импорт библиотек и настройки

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
# pip install catboost

In [60]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool

In [61]:
!pip freeze | grep "numpy\|pandas\|lightgbm\|scikit-learn"

geopandas==0.13.2
lightgbm==4.1.0
numpy==1.25.2
pandas==2.0.3
pandas-datareader==0.10.0
pandas-gbq==0.19.2
pandas-stubs==2.0.3.230814
scikit-learn==1.2.2
sklearn-pandas==2.2.0


In [62]:
pd.options.display.max_rows = 93
pd.options.display.max_columns = 93

## Загрузка данных

In [63]:
train_df = pd.read_parquet("/content/drive/MyDrive/Hakaton/restored_train.pqt") # changed
test_df = pd.read_parquet("/content/drive/MyDrive/Hakaton/test_data.pqt")

## Анализ

Размерность

In [64]:
train_df.shape, test_df.shape

((600000, 93), (290120, 92))

In [65]:
train_df.head()

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.452182,2.736461,2.932560,2.437979,okved_30,segment_1,0.942275,0.536013,0.511490,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.396523,0.893064,0.463405,0.505823,0.873795,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,0.099899,0.226247,0.391439,1.054211,0.654057,0.387856,1.171020,0.648942,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,0.356556,0.426034,1.201541,0.435983,0.864246,1.270517,-0.152107,0.286074,0.891856,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.186820,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.469917,2.756014,2.952725,2.455375,okved_30,segment_1,0.645704,0.536378,0.486425,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.088209,0.969891,-0.063678,0.221993,0.131636,0.893042,0.398889,0.095082,0.873685,0.526503,-0.101797,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.021465,0.224276,0.326923,0.419979,0.652830,0.258824,0.410410,0.633498,0.564854,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,-0.084701,0.307935,0.302226,0.425993,1.135607,0.411205,0.864251,1.237550,-0.142187,0.286081,0.902845,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.125043,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.485936,2.773674,2.970940,2.472771,okved_30,segment_1,0.403604,0.613167,0.480547,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.485924,0.893049,0.560179,0.148894,0.873656,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.029795,0.223291,0.294665,1.251369,0.653444,0.387856,0.552757,0.629637,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,-0.084701

Проверка на дубликаты

In [66]:
train_df.duplicated(keep='first').sum()

0

Проверка на пропуски

In [67]:
train_df.isna().sum()

id                              0
date                            0
balance_amt_avg             65577
balance_amt_max             65577
balance_amt_min             65577
balance_amt_day_avg         65577
channel_code                    0
city                            0
city_type                       0
index_city_code                 0
ogrn_days_end_month         31473
ogrn_days_end_quarter       31473
ogrn_month                      0
ogrn_year                       0
ft_registration_date        31360
max_founderpres            335592
min_founderpres            335592
ogrn_exist_months           31473
okved                           0
segment                         0
sum_of_paym_2m             148884
sum_of_paym_6m             148884
sum_of_paym_1y             148884
sum_a_oper_1m                   0
cnt_a_oper_1m              196181
sum_b_oper_1m                   0
cnt_b_oper_1m              196181
sum_c_oper_1m                   0
cnt_c_oper_1m              196181
sum_deb_d_oper

In [68]:
test_df.isna().sum()

id                              0
date                            0
balance_amt_avg             29483
balance_amt_max             29483
balance_amt_min             29483
balance_amt_day_avg         29483
channel_code                20028
city                        60612
city_type                   60992
index_city_code            156461
ogrn_days_end_month         14278
ogrn_days_end_quarter       14278
ogrn_month                  14278
ogrn_year                   14278
ft_registration_date        14838
max_founderpres            161192
min_founderpres            161192
ogrn_exist_months           14278
okved                       14479
segment                     14020
sum_of_paym_2m              73461
sum_of_paym_6m              73461
sum_of_paym_1y              73461
sum_a_oper_1m                   0
cnt_a_oper_1m               95210
sum_b_oper_1m                   0
cnt_b_oper_1m               95210
sum_c_oper_1m                   0
cnt_c_oper_1m               95210
sum_deb_d_oper

Количество дубликатов

In [69]:
train_df.dropna().shape

(173529, 93)

In [70]:
test_df.dropna().shape

(44938, 92)

Информация

In [71]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 93 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       600000 non-null  int64  
 1   date                     600000 non-null  object 
 2   balance_amt_avg          534423 non-null  float64
 3   balance_amt_max          534423 non-null  float64
 4   balance_amt_min          534423 non-null  float64
 5   balance_amt_day_avg      534423 non-null  float64
 6   channel_code             600000 non-null  object 
 7   city                     600000 non-null  object 
 8   city_type                600000 non-null  object 
 9   index_city_code          600000 non-null  object 
 10  ogrn_days_end_month      568527 non-null  float64
 11  ogrn_days_end_quarter    568527 non-null  float64
 12  ogrn_month               600000 non-null  object 
 13  ogrn_year                600000 non-null  object 
 14  ft_r

Начальные кластеры

In [72]:
test_df['start_cluster'].value_counts()

start_cluster
{α}          115245
{}            28269
{α, η}        13634
{other}       10732
{α, γ}        10339
{α, β}         3629
{α, δ}         2813
{α, ε}         1621
{α, θ}         1380
{α, ψ}          876
{α, μ}          542
{α, ε, η}       413
{α, λ}          258
{α, ε, θ}       232
{α, ε, ψ}        88
{λ}              45
{α, π}            4
Name: count, dtype: int64

In [73]:
train_df[['id', 'date', 'start_cluster', 'end_cluster']].isna().sum()

id               0
date             0
start_cluster    0
end_cluster      0
dtype: int64

## Предобработка

In [74]:
list_of_col_train = train_df.columns[train_df.isna().sum() > 0].tolist()
list_of_col_test = test_df.columns[test_df.isna().sum() > 0].tolist()
list_of_col_train_cat = [
    "channel_code", "city", "city_type",
    "okved", "segment",
    "index_city_code", "ogrn_month", "ogrn_year",
    'start_cluster'
]
list_of_custom_cols = ["balance_amt_avg",
"balance_amt_max",
"balance_amt_min",
"balance_amt_day_avg",
"channel_code",
"city",
"city_type",
"index_city_code",
"ogrn_days_end_month",
"ogrn_days_end_quarter",
"ogrn_month",
"ogrn_year",
"ft_registration_date",
"max_founderpres",
"min_founderpres",
"ogrn_exist_months",
"okved",
"segment",
"sum_of_paym_2m",
"sum_of_paym_6m",
"sum_of_paym_1y"]

list_of_custom_cols_test = ["balance_amt_avg",
"balance_amt_max",
"balance_amt_min",
"balance_amt_day_avg",
"channel_code",
"city",
"city_type",
"index_city_code",
"ogrn_days_end_month",
"ogrn_days_end_quarter",
"ogrn_month",
"ogrn_year",
"ft_registration_date",
"max_founderpres",
"min_founderpres",
"ogrn_exist_months",
"okved",
"segment",
"sum_of_paym_2m",
"sum_of_paym_6m",
"sum_of_paym_1y",
'start_cluster']

In [79]:
def restore_data(df, list_of_col):
    # Копия датасета чтобы не грязнить основной
    restored_df = df.copy()

    # Цикл по всем необходимым столбцам
    # Некоторые столбцы не имеют пропусков, поэтому их даже не трогаю
    # Можно задавать кастомные столбцы, чтоб например обработать только 5 столбцов
    for name_of_col in list_of_col:
      # Вывод информации чтоб понимать на каком мы этапе
      print('Столбец', name_of_col)

      # РАБОТАЕТ ПИЗДЕЦ КАК МЕДЛЕННО, У МЕНЯ БАБУШКА ХОДИТ БЫСТРЕЕ В МАГАЗ
      # Вариант 1 - числовые значения
      if restored_df[name_of_col].dtype in [np.float16, np.int64, np.float64, np.int32, np.int16, np.float32]: # проверка на числовые значения (какой тип данных в столбце)

          # temp_df - сабсет с id, date и name_pf_col, с удаленными пропусками
          temp_df = restored_df[['id','date', name_of_col]].copy().dropna()

          # маска, чтобы оставить в temp_df только те id,
          # которые встречаются 2-3 раза
          # это нужно чтобы считать изменение значения по name_of_col (10 -> 15 = прирост 1.5)
          # с единственным значением не посчитаешь так
          mask = temp_df['id'].value_counts().isin([2, 3])
          temp_df = temp_df[temp_df['id'].isin(mask.index[mask])]

          # Разделение данных по id
          # Как раз для подсчета изменения в каждом случае
          grouped_data = temp_df.groupby('id')
          growth_rates = []

          # Вычисление среднего коэффициента изменения для каждого id
          for id_val, group in grouped_data:
              if len(group) == 3:  # Убедиться, что у нас есть данные для расчета прироста
                  # Вычислить изменения между соседними месяцами
                  growth_rate = ((group[name_of_col].iloc[2] / group[name_of_col].iloc[1]) +
                                (group[name_of_col].iloc[1] / group[name_of_col].iloc[0])) / 2
                  growth_rates.append(growth_rate)

                  # Это когда всего 2 значения
              elif len(group) == 2:
                  growth_rate = group[name_of_col].iloc[1] / group[name_of_col].iloc[0]
                  growth_rates.append(growth_rate)

          # Найти среднее значение средних коэффициентов изменения
          # growth_rates = np.abs(growth_rates) # это не надо, не логично, потому что не учитывает убыль, а только прибыль, короче хуйня не используем

          # среднее изменение столбца (в формате 0.05, что означает в среднем по столбцу разница от месяца к месяцу прирост 5%)
          aver_diff = np.mean(growth_rates)-1


          # Цикл по каждому клиенту (id)
          for id_val in restored_df['id'].unique():

            # Информация для вывода (понимать на каком мы этапе)
            if id_val % 10000 == 0:
              print('Index', id_val)

            # available_months - сабсет из 1-3 значений (зависит от пропусков)
            # в нем 1-3 значения конкретного id, также date и значение name_of_col
            # также пропуски убираются
            available_months = restored_df[restored_df['id'] == id_val].copy()[['date', name_of_col]].dropna(axis=0)

            # Проверка, сколько доступно значений name_of_col (короче сколько не пропущено)
            # если всего 1 значение, значит 2 пропуска нужно восстановить
            if len(available_months[name_of_col].values) == 1:

                # Если у нас доступно только одно значение и оно в month_1
                # Тогда мы берем это значение (first_value)
                # и домножаем по среднему изменению и получаем month_2 и month_3
                if ['month_1'] in available_months['date'].values:
                    first_value = available_months.loc[available_months['date'] == 'month_1', name_of_col].values[0]
                    restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_2'), name_of_col] = first_value * (1+aver_diff)
                    restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_3'), name_of_col] = first_value * (1+aver_diff) * (1+aver_diff)

                # Если у нас доступно только одно значение и оно в month_2
                # Тогда мы берем это значение (first_value)
                # и домножаем по среднему изменению и получаем month_1 и month_3
                elif ['month_2'] in available_months['date'].values:
                    first_value = available_months.loc[available_months['date'] == 'month_2',name_of_col].values[0]
                    restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_1'), name_of_col] = first_value / (1 + aver_diff)
                    restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_3'), name_of_col] = first_value * (1 + aver_diff)

                # Если у нас доступно только одно значение и оно в month_3
                # Тогда мы берем это значение (first_value)
                # и домножаем по среднему изменению и получаем month_1 и month_2
                elif ['month_3'] in available_months['date'].values:
                    first_value = available_months.loc[available_months['date'] == 'month_3', name_of_col].values[0]
                    restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_1'), name_of_col] = first_value / (1 + aver_diff) / (1 + aver_diff)
                    restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_2'), name_of_col] = first_value / (1 + aver_diff)

            # Если доступно изначально 2 значения (1 пропуск заполнить)
            elif len(available_months[name_of_col].values) == 2:

                # Если есть month_1 и month_2, надо найти month_3
                if ('month_1' in available_months['date'].values) and ('month_2' in available_months['date'].values):
                  first_value = available_months.loc[available_months['date'] == 'month_1', name_of_col].values[0]
                  second_value = available_months.loc[available_months['date'] == 'month_2', name_of_col].values[0]
                  restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_3'), name_of_col] = second_value * (second_value/first_value)

                # Если есть month_1 и month_3, надо найти month_2
                elif ('month_1' in available_months['date'].values) and ('month_3' in available_months['date'].values):
                  first_value = available_months.loc[available_months['date'] == 'month_1', name_of_col].values[0]
                  second_value = available_months.loc[available_months['date'] == 'month_3', name_of_col].values[0]
                  restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_2'), name_of_col] = (first_value + second_value)/2

                # Если есть month_2 и month_3, надо найти month_1
                elif ('month_2' in available_months['date'].values) and ('month_3' in available_months['date'].values):
                  first_value = available_months.loc[available_months['date'] == 'month_2', name_of_col].values[0]
                  second_value = available_months.loc[available_months['date'] == 'month_3', name_of_col].values[0]
                  restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_1'), name_of_col] = first_value * (first_value/second_value)

            # Если вообще нету значений ни по какому из месяцев
            elif len(available_months[name_of_col].values) == 0:
                  mean_of_col = temp_df[name_of_col].mean() # берем среднее по стобцу значение
                  var_of_col = temp_df[name_of_col].var() # находим дисперсию
                  first_value = mean_of_col + np.random.uniform(-var_of_col, var_of_col) # получаем среднее значение с рандомным значением в дисперсии
                  # насчет np.random.uniform(-var_of_col, var_of_col) не уверен, берет он одно значение или как...
                  restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_1'), name_of_col] = first_value
                  restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_2'), name_of_col] = first_value * (1 + aver_diff)
                  restored_df.loc[(restored_df['id'] == id_val) & (restored_df['date'] == 'month_3'), name_of_col] = first_value * (1 + aver_diff) * (1+aver_diff)


      else:
        # РАБОТАЕТ, ТУТ ВСЕ ОКЕЙ, НО ЕСЛИ ЕСТЬ ИДЕИ - ДОБАВЛЯЙТЕ
        # Вариант 2 - категориальные значения
          value_counts = restored_df[name_of_col].value_counts(normalize=True) # получаем список возможных значений конкретного столбца и их вероятность появления от 0 до 1
          for id_val in restored_df['id'].unique(): # цикл по всем id (клиентам), которые есть

              # информация для вывода, чтобы понять, на каком мы этапе обработки
              if id_val % 10000 == 0:
                  print('Index', id_val)

              # id_data - сабсет с айди клиента, месяцами и нужным столбцом, который мы обрабатываем
              # также пропуски убираю, дальше увидите почему
              id_data = restored_df[restored_df['id'] == id_val][['id', 'date', name_of_col]].dropna()
              len_id_data = len(id_data)

              # если пропусков нету, значит все значения конкретного столбца name_of_col заполнены
              # а значит пропускаем
              if len_id_data == 3:
                  continue

              # если есть 1 или 2 пропуска, то беру доступное значение с другого месяца и заполняю им пропуски
              elif len_id_data > 0:
                value1 = id_data[name_of_col].values[0]
                restored_df.loc[(restored_df['id'] == id_val), name_of_col] = value1

              # если вообще нет информации о типе, тогда берем value_counts, описанный выше, и рандомно выбираем
              # значение по распределению столбца
              elif len_id_data == 0:
                value2 = np.random.choice(value_counts.index, p=value_counts.values)
                restored_df.loc[(restored_df['id'] == id_val), name_of_col] = value2



    return restored_df

# Usage
# restored_df_train = restore_data(train_df, list_of_custom_cols)
# restored_df_train.to_parquet('restored_train_clear.pqt')

df_test = restore_data(test_df, ['start_cluster'])
# restored_df_train.to_parquet('restored_test.pqt')

Столбец start_cluster
Index 200000
Index 210000
Index 220000
Index 230000
Index 240000
Index 250000
Index 260000
Index 270000
Index 280000
Index 290000


In [84]:
test_df = df_test.copy()

In [96]:
test_df.isna().sum()

id                              0
date                            0
balance_amt_avg             29483
balance_amt_max             29483
balance_amt_min             29483
balance_amt_day_avg         29483
channel_code                20028
city                        60612
city_type                   60992
index_city_code            156461
ogrn_days_end_month         14278
ogrn_days_end_quarter       14278
ogrn_month                  14278
ogrn_year                   14278
ft_registration_date        14838
max_founderpres            161192
min_founderpres            161192
ogrn_exist_months           14278
okved                       14479
segment                     14020
sum_of_paym_2m              73461
sum_of_paym_6m              73461
sum_of_paym_1y              73461
sum_a_oper_1m                   0
cnt_a_oper_1m               95210
sum_b_oper_1m                   0
cnt_b_oper_1m               95210
sum_c_oper_1m                   0
cnt_c_oper_1m               95210
sum_deb_d_oper

In [23]:
train_df.dropna(inplace=True, axis=0)

In [85]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"
]

Обозначение категориальных признаков

In [ ]:
train_df[cat_cols] = train_df[cat_cols].astype("category")
test_df[cat_cols] = test_df[cat_cols].astype("category")

In [106]:
label_encoders = {}

for col in cat_cols:
    label_encoders[col] = LabelEncoder()
    train_df[col] = label_encoders[col].fit_transform(train_df[col])
    test_df[col] = label_encoders[col].fit_transform(test_df[col])

Создаем выборки для валидации и обучения

In [89]:
X = train_df.drop(["id", "date", "end_cluster"], axis=1)
y = train_df["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

## Обучение модели

### CatBoostClassifier

In [90]:
model = CatBoostClassifier(iterations=1024,
                           depth=14,
                           learning_rate=0.01,
                           loss_function='MultiClass',
                           task_type="GPU",
                           devices='0')
model.fit(x_train, y_train)

0:	learn: 2.7481313	total: 1.18s	remaining: 20m 2s
1:	learn: 2.6726266	total: 2.25s	remaining: 19m 11s
2:	learn: 2.6046383	total: 3.32s	remaining: 18m 49s
3:	learn: 2.5431448	total: 4.39s	remaining: 18m 39s
4:	learn: 2.4867552	total: 5.47s	remaining: 18m 33s
5:	learn: 2.4348036	total: 6.55s	remaining: 18m 31s
6:	learn: 2.3864146	total: 7.65s	remaining: 18m 31s
7:	learn: 2.3414263	total: 8.84s	remaining: 18m 43s
8:	learn: 2.2993221	total: 10.1s	remaining: 18m 57s
9:	learn: 2.2597352	total: 11.3s	remaining: 19m 2s
10:	learn: 2.2224594	total: 12.3s	remaining: 18m 56s
11:	learn: 2.1870669	total: 13.4s	remaining: 18m 51s
12:	learn: 2.1534973	total: 14.5s	remaining: 18m 45s
13:	learn: 2.1217039	total: 15.5s	remaining: 18m 41s
14:	learn: 2.0914486	total: 16.6s	remaining: 18m 37s
15:	learn: 2.0625268	total: 17.7s	remaining: 18m 34s
16:	learn: 2.0349095	total: 18.8s	remaining: 18m 32s
17:	learn: 2.0084598	total: 19.9s	remaining: 18m 29s
18:	learn: 1.9829012	total: 20.9s	remaining: 18m 26s
19:	l

### LGBM

В качестве базовой модели возьмем LGBM обучим на всех признаках

In [ ]:
model = LGBMClassifier(verbosity=-1, random_state=42, n_jobs=-1)
model.fit(x_train, y_train)

LGBMClassifier(n_jobs=-1, random_state=42, verbosity=-1)

### Метрики

Зададим функцию для взвешенной метрики roc auc

In [107]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [108]:
cluster_weights = pd.read_excel("/content/drive/MyDrive/Hakaton/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

Проверка работы модели

In [109]:
y_pred_proba = model.predict_proba(x_val)
y_pred_proba.shape

(120000, 17)

In [110]:
weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.9369424931107928

## Прогноз на тестовой выборке

In [111]:
test_df.pivot(index="id", columns="date", values="start_cluster").head(3)

date,month_4,month_5,month_6
id,,,
200000,15.0,15.0,15.0
200001,15.0,15.0,15.0
200002,0.0,0.0,0.0


Для того, чтобы сделать прогноз на тестовой выборке, нужно заполнить стартовый кластер. </br>
В качестве базового подхода заполним все стартовые кластеры, самым популярным кластером.

In [112]:
test_df['start_cluster'].value_counts()

start_cluster
15    170103
1      51842
9      19619
0      15694
3      15209
2       5346
4       4224
8       2463
10      2049
14      1307
12       735
5        612
11       365
6        345
7        126
16        72
13         9
Name: count, dtype: int64

In [ ]:
test_df["start_cluster"] = train_df["start_cluster"].mode()[0]
# test_df["start_cluster"] = test_df["start_cluster"].astype("category")

In [113]:
sample_submission_df = pd.read_csv("/content/drive/MyDrive/Hakaton/sample_submission.csv")

In [114]:
sample_submission_df.shape

(100000, 18)

In [115]:
sample_submission_df.head()

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
1,200001,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,200002,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
3,200003,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
4,200004,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


Для тестовой выборки будем использовать только последний месяц

In [116]:
last_m_test_df = test_df[test_df["date"] == "month_6"]
last_m_test_df = last_m_test_df.drop(["id", "date"], axis=1)

In [117]:
test_pred_proba = model.predict_proba(last_m_test_df)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

In [118]:
test_pred_proba_df.shape

(100000, 17)

In [119]:
test_pred_proba_df.head(2)

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.011769,0.021327,0.014942,0.022427,0.005848,0.001310,0.001883,0.001463,0.013753,0.008138,0.013679,0.003266,0.004647,0.000628,0.002738,0.871542,0.000639
1,0.011775,0.581708,0.001098,0.002226,0.000408,0.000369,0.000173,0.000099,0.001020,0.012333,0.001118,0.000134,0.000776,0.000058,0.000493,0.386097,0.000116


In [120]:
sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("catboost_submission_3.csv", index=False)

In [121]:
sample_submission_df

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.011769,0.021327,0.014942,0.022427,0.005848,0.001310,0.001883,0.001463,0.013753,0.008138,0.013679,0.003266,0.004647,0.000628,0.002738,0.871542,0.000639
1,200001,0.011775,0.581708,0.001098,0.002226,0.000408,0.000369,0.000173,0.000099,0.001020,0.012333,0.001118,0.000134,0.000776,0.000058,0.000493,0.386097,0.000116
2,200002,0.666618,0.007002,0.011535,0.089499,0.017469,0.004262,0.005124,0.006864,0.034667,0.009884,0.017948,0.005467,0.003886,0.001323,0.023924,0.092985,0.001544
3,200003,0.029781,0.648949,0.001060,0.001988,0.000849,0.000531,0.000290,0.000095,0.000911,0.019734,0.002360,0.000155,0.001162,0.000078,0.000625,0.291354,0.000078
4,200004,0.123171,0.137585,0.129571,0.041198,0.004997,0.004769,0.002218,0.001383,0.016207,0.056239,0.015154,0.002235,0.007326,0.000901,0.003773,0.452069,0.001203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.016067,0.264709,0.002921,0.005345,0.001397,0.000151,0.000496,0.000128,0.002496,0.001329,0.002645,0.001568,0.000326,0.000211,0.001765,0.698072,0.000375
99996,299996,0.019997,0.032065,0.011459,0.038284,0.006253,0.000342,0.001118,0.000426,0.007560,0.004986,0.006216,0.001403,0.001483,0.000127,0.002636,0.865419,0.000224
99997,299997,0.027576,0.027434,0.018672,0.070488,0.016211,0.000676,0.001090,0.000519,0.014239,0.007665,0.006222,0.000967,0.002917,0.000372,0.007950,0.796831,0.000169
99998,299998,0.094955,0.173208,0.017352,0.038583,0.003777,0.001846,0.001506,0.000886,0.010600,0.015913,0.008916,0.003097,0.002490,0.000663,0.005205,0.619849,0.001154
